In [1]:
import dataflowkit.datasets as D
import dataflowkit.graphs as G
import dataflowkit.recipes as R
from dataflowkit.utils.print_time import print_time, print_end
from multiprocessing import Process, Queue, cpu_count
import threading, numpy as np

In [2]:
class DummyRecipe1(R.BaseRecipe):
    # Must be executed in singleton, maybe heavy memory demanding
    def execute(self, ins, outs):
        # Takes ~ 0.093899s
        for i in np.arange(1e6):
            pass

In [3]:
class DummyRecipe2(R.BaseRecipe):
    # Hevay calcuation process
    def execute(self, ins, outs):
        # Takes ~ 0.823933s
        for i in np.arange(1e7):
            pass

In [4]:
def thread_job(file_queue, mrf, master_index):
    while not file_queue.empty():
        file = file_queue.get()
        print_time('master_index: ' + str(master_index) + ' file: ' + str(file))
        
        # declare recipes
        dummy_recipe1 = DummyRecipe1()
        dummy_recipe2 = DummyRecipe2()
        
        # declare datasets
        dataset1 = D.InMemory()
        dataset2 = D.InMemory()
        dataset3 = D.InMemory()
        
        # declare graph
        dummy_recipe1.execute(ins=dict(dataset1=dataset1), outs=dict(dataset2=dataset2))
        dummy_recipe2.execute(ins=dict(dataset2=dataset2), outs=dict(dataset3=dataset3))
        
        print_end('master_index: ' + str(master_index) + ' file: ' + str(file))

#### Declare the files be process

In [5]:
file_queue = Queue()
files = np.arange(20)
for file in files:
    file_queue.put(file)

#### MasterRecipeFactory must be initialized before opeining the processes

In [6]:
no_master = 4
mrf = G.MasterRecipeFactory.get_inatance()

#### Declare the threads

In [7]:
threads = [threading.Thread(target=thread_job, args=(file_queue, mrf, master_index)) 
           for master_index in np.arange(no_master)]

#### Execute the threads

In [8]:
for t in threads:
    t.start()
for t in threads:
    t.join()

master_index: 1 file: 1 2.673495
master_index: 3 file: 3 2.779977
master_index: 2 file: 2 2.977495
master_index: 0 file: 0 3.067415
master_index: 1 file: 4 2.327544
master_index: 3 file: 5 2.278266
master_index: 2 file: 6 2.140716
master_index: 0 file: 7 2.41691
master_index: 1 file: 8 2.393383
master_index: 3 file: 9 2.358779
master_index: 2 file: 10 2.471605
master_index: 0 file: 11 2.320105
master_index: 1 file: 12 2.212133
master_index: 2 file: 14 2.407975
master_index: 3 file: 13 2.648249
master_index: 0 file: 15 2.422721
master_index: 1 file: 16 2.62666
master_index: 3 file: 18 2.292151
master_index: 2 file: 17 2.451919
master_index: 0 file: 19 2.204667
